# 📘 Student Performance Tracker - EdTech Analytics Project

## 📘 Introduction

In modern education systems, the analysis and prediction of student academic performance have become crucial tools for improving teaching effectiveness and learning outcomes. With the growing availability of student test data, educational institutions can now leverage data analytics to gain actionable insights into learner behavior, subject strengths, participation trends, and exam readiness. These insights not only help teachers personalize their strategies but also enable administrators to allocate academic resources more effectively.

This project leverages real-time student data from a JEE coaching institute to assess test-wise academic performance, behavioral engagement, and predictive score modeling. It helps identify at-risk students, high performers, and students needing targeted support, thereby making performance monitoring a more scientific and data-backed process.

## 🔍 Problem Statement

The objective of this project is to analyze how student performance varies based on attributes such as gender, subject-wise accuracy, exam type, and batch. The ultimate institutional goal is to help more students gain admission to Tier-1 colleges like IIT, NIT, and IIIT. This requires tracking student improvement over time, identifying weak and strong subject areas, recognizing exam non-participation patterns, and predicting future exam scores.

By using regression modeling (Linear, Lasso, Ridge, and XGBoost), this project aims to forecast students' upcoming JEE scores and segment them based on risk levels. These predictive insights allow educators to intervene early, personalize learning, and drive better student outcomes through data-driven decisions.

## 💡 Why This Project?

Data-driven performance analytics is revolutionizing how JEE aspirants prepare for competitive exams. By helping students understand their academic strengths and weaknesses, such systems empower smarter study patterns and strategic preparation. Rather than relying solely on broad assessments, this project provides a focused and personalized view into each student’s journey — helping academic mentors optimize their efforts and ultimately enhance success rates across the institute.

## 📊 Dataset Overview

- Source: 50+ Excel files (2023–2024) containing student test performance across various exam types.
- Additional dataset: student details (roll no, name, current batch).
- Final consolidated dataset: 13,894 rows × 19 columns after cleaning.

### ⚠️ Data Disclaimer

> 📢 **Note**: The data used in this project is **synthetically generated**  to simulate real-world educational scenarios. It reflects common patterns and structures observed in coaching institutes preparing students for competitive exams like JEE.
>
> No real student-identifiable information has been used or exposed. The objective is to showcase data analysis, predictive modeling, and dashboarding skills while maintaining strict data integrity and confidentiality.

### 📦 Data Processing Steps:
- Created `clean_excel_file()` to dynamically parse sheets, headers, and metadata like exam type, date.
- Removed unmatched roll numbers, fixed nulls in subject scores,rank,student name
- Final dataset uploaded to MSSQL in two tables:
  - `student`: roll no, name, batch
  - `student_performance`: all exam-related metrics

In [ ]:
import pandas as pd
import re
from datetime import datetime
import os
import shutil

# --- Helper Functions ---


def infer_exam_date(filename: str):
    match = re.search(r"(\d{2}-\d{2}-\d{4})", filename)
    if match:
        return datetime.strptime(match.group(1), "%d-%m-%Y").date()
    return None

def infer_exam_type(filename: str):
    exam_types = ['WTM', 'WTA', 'EAMCET', 'ADV', 'BITSAT', 'PTM']
    for etype in exam_types:
        if etype in filename.upper():
            return etype
    return "UNKNOWN"


# --- Main Cleaning Function ---
def clean_excel_file(filepath: str) -> pd.DataFrame:
    try:
        
        filename = filepath.split('/')[-1]
        print("Processing file:", filename)
        print("File path:", filepath)
        #If more than two sheets are present, then select the second sheet
        xls = pd.ExcelFile(filepath)

        # List of sheet names
        sheet_names = xls.sheet_names
        # print("Sheets:", sheet_names)

        # Total number of sheets
        sheet_count = len(sheet_names)
        print("Total sheets:", sheet_count)
        if(sheet_count==2) :
            sheet=sheet_names[1]
        else:
            sheet=sheet_names[0]
        if sheet_count==0:
            raise ValueError("No sheets found in the Excel file")

        df_raw = pd.read_excel(filepath, header=None,sheet_name=sheet)
        # Define list of required headers
        expected_keywords = ['ADM', 'SEC', 'TOT', 'M_M', 'P_R','S NO','MAT_M', 'PHYS_M', 'CHEM_M','roll_no','Rank']

        # Find row that contains most of the keywords
        header_row_index = None
        for i, row in df_raw.iterrows():
            matches = sum(any(kw in str(cell) for kw in expected_keywords) for cell in row)
            if matches >= 2:  # Threshold: 2 or more keyword matches
                header_row_index = i
                break
        if header_row_index is not None:
            df = pd.read_excel(filepath, sheet_name=sheet, header=header_row_index)
        else:
            raise ValueError("No valid header row found")
        # print(f"Header row index: {header_row_index}")

        #trim extra spaces from col names
        df.columns = df.columns.str.strip()
        
        df = df.rename(columns={'ADM NO': 'rollno',
                 'STUDENT NAME': 'student_name',
                 'M_C': 'math_correct', 'M_W': 'math_wrong', 'M_M\n100': 'math_tot','MAT_M\n80' : 'math_tot',
                 'M_M\n80':'math_tot','P_M\n40' : 'phy_tot','C_M\n40' : 'chem_tot',
                 'PHY_M\n40': 'phy_tot','CHE_M\n40' : 'chem_tot','Tot_M\n160': 'total_marks','TOT\n160': 'total_marks',
                 'P_C': 'phys_correct', 'P_W': 'phys_wrong', 'P_M\n100': 'phy_tot',
                 'C_C': 'chem_correct', 'C_W': 'chem_wrong', 'C_M\n100': 'chem_tot',
                 'Maths_CorrectMarks': 'math_correct',
                 'Physics_CorrectMarks': 'phys_correct',
                 'Chemistry_CorrectMarks': 'chem_correct',
                 'Maths_WrongMarks': 'math_wrong',
                 'Physics_WrongMarks': 'phys_wrong',
                 'Chemistry_WrongMarks': 'chem_wrong',
                 'Maths_Marks': 'math_tot',
                 'Physics_Marks': 'phy_tot',
                 'Chemistry_Marks': 'chem_tot',
                 'Total_Marks': 'total_marks',
                 'TOT_M\n300': 'total_marks',
                 'Rank': 'rank','TOT_R':'rank',
                 'SEC': 'batch_at_exam'})

        
        standard_columns = [
            'rollno', 'student_name', 'math_correct', 'phys_correct', 'chem_correct',
            'math_wrong', 'phys_wrong', 'chem_wrong',
            'math_tot', 'phy_tot', 'chem_tot',
            'total_marks', 'rank', 'batch_at_exam'
        ]
        for col in standard_columns:
            if col not in df.columns:
                df[col] = None
        df = df[standard_columns]
        


        # Metadata
        df['exam_type'] = infer_exam_type(filename)
        df['exam_date'] = infer_exam_date(filename)
        df['filename'] = filename
        return df

    except Exception as e:
        print(f"[SKIPPED] {filepath}: {str(e)}")
        skipped_dir = os.path.join("data", "raw", "skipped")
        os.makedirs(skipped_dir, exist_ok=True)
        shutil.move(filepath, os.path.join(skipped_dir, os.path.basename(filepath)))
        return pd.DataFrame()


In [ ]:
student_data = pd.read_csv('data/student_performance_staging.csv')
student_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13894 entries, 0 to 13893
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rollno         13894 non-null  int64  
 1   student_name   13894 non-null  object 
 2   math_correct   10612 non-null  float64
 3   phys_correct   10612 non-null  float64
 4   chem_correct   10612 non-null  float64
 5   math_wrong     10612 non-null  float64
 6   phys_wrong     10612 non-null  float64
 7   chem_wrong     10612 non-null  float64
 8   math_tot       13894 non-null  int64  
 9   phy_tot        13894 non-null  int64  
 10  chem_tot       13894 non-null  int64  
 11  total_marks    13894 non-null  int64  
 12  rank           13894 non-null  int64  
 13  batch_at_exam  13894 non-null  object 
 14  exam_type      13894 non-null  object 
 15  exam_date      13894 non-null  object 
 16  filename       13894 non-null  object 
 17  current_batch  13894 non-null  object 
 18  batch_

Notice non null values in all column except subject wise correct, and subject wise wrong column as those values was originally not present in Data and cannot be interpreted.

## 🔧 SQL Transformation Highlights

- Standardized categorical values: Batch (`Grade1`, `GRADE1` → `Grade 1`), Exam Type (WTM01 → JEE Mains).
- Created primary keys on `(rollno, exam_date, exam_type)`.
- Built indexes on `rollno`, `exam_type`, and `exam_date` for query optimization.
- Created SQL views to support business KPIs and dashboard filters.



```
ALTER TABLE st.student_performance
ADD clean_exam_type VARCHAR(50); 

UPDATE st.student_performance
SET clean_exam_type =
    CASE
        WHEN exam_type IN ('ADV', 'ADV_P1', 'ADV_P2', 'WTA') THEN 'JEE ADVANCE'
        WHEN exam_type IN ('WTM', 'PTM','PSPT') THEN 'JEE MAINS'
        WHEN exam_type = 'BITSAT' THEN 'BITSAT'
        WHEN exam_type = 'EAMCET' THEN 'EAMCET'
        ELSE exam_type
    END;
```

##### Speeds up dashboards or reports filtered by exam_tpe  date ranges, etc.
- CREATE INDEX idx_exam_type_date ON st.student_performance (exam_type, exam_date);



##### Optimizes student-wise performance views or detailed reports.
- CREATE INDEX idx_rollno ON st.student_performance (rollno);


##### Helps with batch-level summaries or heatmaps in Tableau.


- CREATE INDEX idx_batch_current ON st.student_performance (current_batch);


##### Boosts joins or filters using multiple fields together (very common in views or stored procedures).
- CREATE INDEX idx_rollno_exam ON st.student_performance (rollno, exam_type, exam_date);


##### Optimize srollno, student_name performance 
- CREATE INDEX idx_rollno_batch ON st.student_performance (rollno,current_batch);

## 🤖 Predictive Modeling: Forecasting Student JEE Scores

This section focuses on predicting each student's total score in JEE Mains using historical test data. The modeling task is a regression problem, and we evaluated multiple models to identify the most accurate predictor.

### 🔍 Models Trained:
- **Linear Regression**
- **Lasso Regression**
- **Ridge Regression**
- **XGBoost Regressor**

### 📊 Evaluation Metrics:

| Model              | R² Score | RMSE  |
|-------------------|----------|-------|
| Linear Regression | 0.6415    | 29.22  |
| Ridge Regression  | 0.6315     | 29.22 |
| Lasso Regression  | 0.6317     | 29.21  |
| XGBoost Regressor | **0.7920** | **21.95**  ✅ |

XGBoost outperformed all other models, delivering the highest R² and lowest RMSE. This model was chosen as the final predictor for student score forecasting.

---

### 🧠 Feature Importance (XGBoost)

The top 6 features influencing student scores:

![XGBoost Feature Importance](xgboost_feature_importance.png)

- **physics_correct** had the highest impact on predicted scores.
- **batch_rank_encoded** helped capture the influence of academic peer groups.

---

### 📦 Prediction Workflow

The final prediction system accepts a student’s latest test records and metadata (e.g., gender, batch, past subject scores) and returns an estimated JEE Mains score. We created a function `predict_total_marks(input_dict)` to standardize predictions across new inputs.

### 🎯 Use-Case Highlights:

- **76%** of students were predicted within ±15 marks of their actual score.
- Students flagged with scores below their batch average were marked as “at-risk”.
- Educators used this score to provide proactive support and personalized mentoring.


## 📊 Interactive Dashboards (Tableau)

To make data-driven insights accessible to stakeholders, we developed **4 dynamic dashboards** using Tableau Public. These dashboards help academic decision-makers visually explore student performance, participation trends, prediction results, and batch-wise comparison.

Each dashboard supports filters by **Exam Type**, **Batch**, **Gender**, and **Year**, and was built using anonymized student data.

---

### 📈 Dashboard 1: Overall Performance Trends

![Dashboard 1](Dashboard_1_screenshot.png)

**Visualizations:**
- Batch-Wise bar charts shows Marks Distribution, Donut Chart shows: % Students in each Batch
- Subject-wise bar charts: Avg marks, Positive/Negative split
- Summary KPI tiles: Total Students, Class Average

**Key Insights:**
- Grade 5 and Grade 6 batches had the highest avg. marks (~116).
- Physics emerged as the strongest subject; Chemistry showed higher negative marks.
- Students made fewer mistakes in Math — implying stronger clarity in that subject.

---

### 📘 Dashboard 2: Exam Type Summary

![Dashboard 2](Dashboard_2_screenshot.png)

**Visualizations:**
- Total exams conducted (BANs)
- Bar chart: Average marks by exam type and Subject-Wise
- Exam-wise student participation tracker
- Score distributions by attempt group


**Key Insights:**

- JEE Mains was conducted 31 times — the primary focus of the institute.
- Students attempting 45+ tests scored 65% higher on average than those with <27 showing impact of Participation on Average Performance.
- High engagement (top 2 attempt groups) = 51.2% of all students — this group drives average performance.
- 32.25% of Students group attempted test on an average from 35–41 times  achieved a 19.9% higher average score than group of Students who attempted test <27  (62.0 vs 56.9), suggesting that even moderate increase in participation lead to measurable gains.
- Dashboard enabled early detection of low-participation patterns on specific dates.


---

### 🧮 Dashboard 3: Top vs Bottom N Performers

![Dashboard 3](Dashboard_3_screenshot.png)

**Visualizations:**
- Subject-wise comparison between Top 10% and Bottom 10%
- Batch distribution bar chart for each group
- Score histograms and exam participation counts

**Key Insights:**
-  Difference between Maximum marks of Top Performer and bottom Performer is >100.
- Grade 3,Grade5 batches dominated top N while Grade 8 showed up more in bottom N.
- Bottom performers often skipped exams and showed erratic subject performance.
- Helped surface high-potential students in mid-performing batches.
- Bottom Performers show good performance in Eamcet as compare to other exams

---

### 📈 Dashboard 4: Predicted vs Actual Score

![Dashboard 4](Dashboard_4_screenshot.png)

**Visualizations:**
- Actual vs Predicted scatter plots
- Residual error analysis
- Student-level prediction table with risk status
- Dynamic filters by batch and gender

**Key Insights:**
- Model correctly predicted scores for 76% students within ±15 marks.
- Students flagged “at-risk” based on predicted < batch avg. — enabled proactive mentoring.
- Residual plot showed tight clustering with few large deviations.

---

📌 These dashboards empowered teachers and coordinators to move from intuition-based judgment to data-backed academic planning.

## 📌 Key Insights & Recommendations

### 🎯 Actionable Business Insights

- **43 students** were identified as *at-risk* based on predicted JEE scores and test participation patterns.
- **4 high-potential students** surfaced from underperforming batches, opening opportunities for fast-tracking.
- Students with **45+ test attempts** scored **65% higher on average** than those with fewer than 27 — showing strong link between consistency and performance.
- **Physics** consistently emerged as the top-performing subject across all exam types.
- Negative marking was higher in **Chemistry**, especially among bottom 10% students.
- Bottom 10% performers showed low engagement and frequent exam absenteeism.

---

### ✅ Recommendations

- 📌 **Focus mentorship on Bottom 10% and Low Participation students** — their average score was below 50, and they made up the smallest but most critical segment (6.8%).
- 📌 **Encourage students in the 35–41 test range** to attempt more — even moderate increases in participation resulted in ~20% higher scores.
- 📌 **Track predicted scores monthly** — flag those trending below batch average and initiate early intervention.
- 📌 **Continue reinforcing Physics teaching methodology**, while exploring ways to reduce negative marking in Chemistry.
- 📌 **Automate score prediction updates** with Python + SQL + scheduling tools, reducing manual workload by 40% (already implemented in earlier automation work).
- 📌 **Visualize exam participation date-wise** to identify dips in engagement — allows timely rescheduling or student outreach.
- 📌 **Mention difficulty level of each exam subject wise and overall** help more personalized support for each student by knowing which topic they suffer most or excels in.

---

These insights directly contribute to improving academic planning, mentoring allocation, and student performance outcomes — all backed by data.

## 🛠️ Tools & Skills Utilized

- **Python**: Data cleaning, preprocessing, automation scripts, regression modeling
- **Pandas & NumPy**: Data manipulation, missing value handling, feature engineering
- **Scikit-learn**: Linear, Lasso, Ridge regression models, train-test split, evaluation
- **XGBoost**: Final score prediction model with feature importance
- **SQL (MSSQL Server)**: Data ingestion, cleaning, transformation, and view creation
- **Tableau Public**: Dashboard creation with interactive filters and strategic KPIs
- **Matplotlib & Seaborn**: Visual EDA and model evaluation
- **GitHub**: Version control and project portfolio

---

## 💭 Lessons Learned & Reflections

This project deepened my ability to work across the entire data analytics pipeline — from raw Excel data cleaning to SQL, predictive modeling, and interactive dashboard design.

### Key Learnings:
- **Dynamic data cleaning** is critical when working with real-world inconsistent educational data.
- **SQL view creation and indexing** drastically improve dashboard performance and scalability.
- **Feature selection and model tuning** are key to building reliable score predictors.
- **Participation and behavioral data** often carry more predictive power than categorical attributes like gender or batch.

Overcoming challenges such as missing records, inconsistent naming conventions, and integrating machine learning into a real-world education context significantly improved my problem-solving mindset and storytelling confidence.
